### # Abrir o arquivo CSV das Bikes extraídos do Kaggle e disponível em Workspace/default/volumes

In [ ]:
# Caminho do arquivo
file_path = '/Volumes/workspace/default/arquivos-aula/Bike Store/products.csv'

# Leitura do arquivo CSV
df_bike = spark.read.format('csv') \
   .option('header', 'true') \
   .option('inferSchema', 'true') \
   .load(file_path)

# Exibe o DataFrame
display(df_bike)

# Mostra o schema detalhado
df_bike.printSchema()

# Mostra em formato de tabela (nome da coluna e tipo)
df_bike.dtypes

### # Cria a Database

# CREATE DATABASE IF NOT EXISTS workspace.db_bikes;

In [ ]:
### # Cria a tabela Produtos

USE CATALOG workspace;
USE SCHEMA db_bikes;
CREATE OR REPLACE TABLE Produtos (

product_id INT,
product_name STRING,
brand_id INT,
category_id INT,
model_year INT,
list_price DOUBLE

);

### # Copia os dados do CSV para dentro da tabela criada

In [ ]:
USE SCHEMA db_bikes;
COPY INTO produtos
FROM '/Volumes/workspace/default/arquivos-aula/Bike Store/products.csv' -- Caminho para o arquivo CSV
FILEFORMAT = CSV -- Formato do arquivo (CSV)
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true'); -- Primeira linha como cabeçalho / Inferir automaticamente o tipo dos dados das colunas

### # Verifica se deu certo selecionando tudo na tabela

In [ ]:
SELECT * FROM produtos;
SELECT product_name as Produto, list_price as Preco FROM db_bikes.produtos WHERE list_price >= '999.99';

### # Primeiro é necessário criar uma tabela temporária para depois criar uma tabela física permanente

In [ ]:
USE db_bikes;

-- Cria tabelas temporárias
CREATE OR REPLACE TEMP VIEW brands_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/brands.csv', header 'true', inferSchema 'true');
CREATE OR REPLACE TEMP VIEW categories_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/categories.csv', header 'true', inferSchema 'true');
CREATE OR REPLACE TEMP VIEW clientes_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/customers.csv', header 'true', inferSchema 'true');
CREATE OR REPLACE TEMP VIEW order_items_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/order_items.csv', header 'true', inferSchema 'true');
CREATE OR REPLACE TEMP VIEW orders_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/orders.csv', header 'true', inferSchema 'true');
CREATE OR REPLACE TEMP VIEW products_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/products.csv', header 'true', inferSchema 'true');
CREATE OR REPLACE TEMP VIEW staffs_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/staffs.csv', header 'true', inferSchema 'true');
CREATE OR REPLACE TEMP VIEW stocks_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/stocks.csv', header 'true', inferSchema 'true');
CREATE OR REPLACE TEMP VIEW stores_tmp USING CSV OPTIONS (path '/Volumes/workspace/default/arquivos-aula/Bike Store/stores.csv', header 'true', inferSchema 'true');

-- Cria as tabelas permanentes a partir das tabelas temporárias
CREATE OR REPLACE TABLE db_bikes.brands AS SELECT * FROM brands_tmp;
CREATE OR REPLACE TABLE db_bikes.categories AS SELECT * FROM categories_tmp;
CREATE OR REPLACE TABLE db_bikes.clientes AS SELECT * FROM clientes_tmp;
CREATE OR REPLACE TABLE db_bikes.order_items AS SELECT * FROM order_items_tmp;
CREATE OR REPLACE TABLE db_bikes.orders AS SELECT * FROM orders_tmp;
CREATE OR REPLACE TABLE db_bikes.products AS SELECT * FROM products_tmp;
CREATE OR REPLACE TABLE db_bikes.staffs AS SELECT * FROM staffs_tmp;
CREATE OR REPLACE TABLE db_bikes.stocks AS SELECT * FROM stocks_tmp;
CREATE OR REPLACE TABLE db_bikes.stores AS SELECT * FROM stores_tmp;


USE db_bikes;

-- Consulta de contratos de clientes.
SELECT customer_id as Id,
first_name as Nome,
phone as Telefone,
email as Email
FROM clientes WHERE first_name like "G%"; -- Filtra os nomes que começam com a letra G.

### # Utilizando o Spark para ler Query SQL (apenas ler, ainda não salva em um Data Frame):

In [ ]:
spark.sql("""

SELECT
customer_id as Id,
first_name as Nome,
phone as Telefone,
email as Email
FROM clientes
WHERE first_name like "G%" -- Filtra os nomes que começam com a letra G.

"""
).show()

### # Salvando a Query SQL num Data Frame: 

In [ ]:
dfSQL = spark.sql("""     -- Dá um nome para o Data Frame (neste exemplo é dfSQL), e remove o .show() que servia para visualizar os dados selecionados.

SELECT
customer_id as Id,
first_name as Nome,
phone as Telefone,
email as Email
FROM clientes
WHERE first_name like "G%"

"""
)

### # Visualizando o Data Frame criado:

In [ ]:
dfSQL.show() # Exibe o Data Frame criado.

# Também funciona com display(dfSQL)

spark.sql(''' 

SELECT * FROM clientes


'''

).show()